# Comparing KNN and ANN search techniques

Let's take a look at how the semantic search works. We will consider KNN and Ann searches with Euclidean and Cosine similarity metrics both. This notebook will find the most similar sentence to the given query from the list of sentences.

Loading necessary libraries:


In [4]:
!pip install sentence-transformers==2.5.1
!pip install annoy==1.17.3
!pip install faiss-cpu==1.8.0

Importing libraries and defining the sentences:

In [25]:
# Import necessary libraries
from sentence_transformers import SentenceTransformer
from sklearn.neighbors import NearestNeighbors
import numpy as np
import faiss

# Load the BAAI model for embeddings
model = SentenceTransformer('BAAI/bge-small-en-v1.5')

# Define sentences
sentences = [
    "Assessing the functionality of different embedding models.",
    "Two approaches for creating embeddings are provided here.",
    "Evaluating the performance of various types of embeddings.",
    "Here, we are presented with two methods for generating embeddings.",
    "Vector search leverages the power of LLMs to provide more relevant and context-aware search results",
    "Vector search significantly improves the user's ability to find information.",
    "K-nearest neighbors is the exact search, which uses a brute force method to find nearest neighbors.",
    "Approximate nearest neighbors trades accuracy for speed gains.",
    "How to measure if two vectors are similar?",
    "The most popular techniques are Euclidean and cosine metrics.",
    "Euclidean Distance measures the physical distance between vectors.",
    "The lower the Euclidean distance, the more similar the items are.",
    "If the Euclidean distance is 0, the points are identical.",
    "As the Euclidean distance increases, the points are considered to be less similar.",
    "Cosine Similarity measures the angle between vectors, focusing on the direction rather than the magnitude.",
    "The higher the cosine similarity, the more similar the items are.",
    "Cosine similarity ranges from -1 to 1, where 1 means the vectors are identical."
]

# Generate embeddings
embeddings = model.encode(sentences)

## KNN Search
The knn_search function performs a k-nearest neighbors search to find the most similar sentences to a given query sentence within a list of sentences. It first encodes the query sentence into an embedding using a pre-trained model. Then, it applies the KNN algorithm, using the specified distance metric, to identify the nearest neighbors among the pre-computed embeddings of the sentences. The function prints the most similar sentence to the query, excluding the query itself if it appears in the sentence list, along with the distance indicating how similar they are. The search considers the `n_neighbors=5` closest neighbors but stops after identifying the first most relevant sentence.

In [63]:
# Function to perform KNN search
def knn_search(query, embeddings, sentences, metric):
    # Generate the embedding for the query sentence
    query_embedding = model.encode([query])
    # Use KNN to find the most similar sentence
    knn = NearestNeighbors(n_neighbors=2, metric=metric)
    knn.fit(embeddings)
    distances, indices = knn.kneighbors(query_embedding)
    for i, index in enumerate(indices[0]):
        if i == 0 and query in sentences:  # Skip the query itself if it's in the list
            continue
        if metric == 'cosine':
            print(
                f"Most similar sentence: {sentences[index]} (Distance: {1-distances[0][i]})")
        else:
            print(
                f"Most similar sentence: {sentences[index]} (Distance: {distances[0][i]})")
        break  # Break after finding the first relevant result

Sometimes the number calculated for the cosine similarity actually presents `1-cosine_similarity`. That's why here we print the value `1-distance`, so it is compatible with this number for ANN search.

## ANN Search and FAISS
The ann_search function conducts an ANN search to identify the most similar sentence to a given query from a list of sentences. It leverages FAISS to perform the search with the choice of either Euclidean or cosine distance metrics.

For cosine similarity, it normalizes both the embeddings and the query. The function then adds the embeddings to a FAISS index, conducts the search, and prints the closest sentence and its associated distance.

In [76]:
# Function to perform ANN search
def ann_search(query, embeddings, sentences, metric):
    k = 2
    dimension = embeddings.shape[1]

    # Selecting the index type based on the metric
    if metric == 'euclidean':
        index = faiss.IndexFlatL2(dimension)
    elif metric == 'cosine':
        # Using inner product for cosine similarity
        index = faiss.IndexFlatIP(dimension)

    assert index.is_trained
    index.add(embeddings)  # Adding the embeddings to the index

    # Normalize query embedding for cosine similarity
    query_embedding = model.encode([query]).astype(np.float32)
    if metric == 'cosine':
        faiss.normalize_L2(query_embedding)

    # Perform the search
    distances, indices = index.search(query_embedding, k)

    for i, idx in enumerate(indices[0]):
        # Skip the query itself if it's in the list
        if i == 0 and metric == 'cosine' and sentences[idx] == query:
            continue
        print(
            f"Most similar sentence: {sentences[idx]} (Distance: {distances[0][i]})")
        break  # Break after finding the first relevant result

Let's take a look at the final result:


In [75]:
query = "Our next subject will be Vector Databases."
print(f"Query: {query}")
print("---------------------------------------------")
# Perform and display results with Euclidean distance
print("KNN Search")
print("\nEuclidean Distance Results:")
knn_search(query, embeddings, sentences, 'euclidean')

# Perform and display results with Cosine distance
print("\nCosine Distance Results:")
knn_search(query, embeddings, sentences, 'cosine')

print("---------------------------------------------")
print("ANN Search")
# Perform and display results with Euclidean distance
print("\nEuclidean Distance Results:")
ann_search(query, embeddings, sentences, 'euclidean')

# Perform and display results with Cosine distance
print("\nCosine Distance Results:")
ann_search(query, embeddings, sentences, 'cosine')

print("---------------------------------------------")

Query: Our next subject will be Vector Databases.
---------------------------------------------
KNN Search

Euclidean Distance Results:
Most similar sentence: Vector search significantly improves the user's ability to find information. (Distance: 0.7374712421176788)

Cosine Distance Results:
Most similar sentence: Vector search significantly improves the user's ability to find information. (Distance: 0.7280680537223816)
---------------------------------------------
ANN Search

Euclidean Distance Results:
Most similar sentence: Vector search significantly improves the user's ability to find information. (Distance: 0.5438637733459473)

Cosine Distance Results:
Most similar sentence: Vector search significantly improves the user's ability to find information. (Distance: 0.7280681729316711)
---------------------------------------------


All methods (KNN and ANN, using both Euclidean and cosine distances) identified the same sentence as the most similar, suggesting a strong semantic connection between the query and this particular sentence.
* **Euclidean Distance:** The distance values differ between KNN and ANN, which is expected as ANN provides an approximation.
* **Cosine Distance:** The distances for cosine similarity are nearly identical across KNN and ANN, suggesting that for this metric and query, the ANN approximation closely matches the exact KNN result.